In [1]:
import pickle
import pandas as pd
import datetime
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import applications
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from DataGen_BinaryLabels import DataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import backend
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, f1_score

class Snapshot(Callback):

    def __init__(self, folder_path, nb_epochs, nb_cycles=5, verbose=0):
        if nb_cycles > nb_epochs:
            raise ValueError('nb_epochs has to be lower than nb_cycles.')

        super(Snapshot, self).__init__()
        self.verbose = verbose
        self.folder_path = folder_path
        self.nb_epochs = nb_epochs
        self.nb_cycles = nb_cycles
        self.period = self.nb_epochs // self.nb_cycles
        self.nb_digits = len(str(self.nb_cycles))
        self.path_format = os.path.join(self.folder_path, 'weights_cycle_{}.h5')


    def on_epoch_end(self, epoch, logs=None):
        if epoch == 0 or (epoch + 1) % self.period != 0: return
        # Only save at the end of a cycle, a not at the beginning

        if not os.path.exists(self.folder_path):
            os.makedirs(self.folder_path)

        cycle = int(epoch / self.period)
        cycle_str = str(cycle).rjust(self.nb_digits, '0')
        self.model.save_weights(self.path_format.format(cycle_str), overwrite=True)

        # Resetting the learning rate
        K.set_value(self.model.optimizer.lr, self.base_lr)

        if self.verbose > 0:
            print('\nEpoch %05d: Reached %d-th cycle, saving model.' % (epoch, cycle))


    def on_epoch_begin(self, epoch, logs=None):
        if epoch <= 0: return

        lr = self.schedule(epoch)
        K.set_value(self.model.optimizer.lr, lr)

        if self.verbose > 0:
            print('\nEpoch %05d: Snapchot modifying learning '
                  'rate to %s.' % (epoch + 1, lr))


    def set_model(self, model):
        self.model = model
        if not hasattr(self.model.optimizer, 'lr'):
            raise ValueError('Optimizer must have a "lr" attribute.')

        # Get initial learning rate
        self.base_lr = float(K.get_value(self.model.optimizer.lr))


    def schedule(self, epoch):
        lr = math.pi * (epoch % self.period) / self.period
        lr = self.base_lr / 2 * (math.cos(lr) + 1)

#returns a list  of  hyperparamter settings
def load_hyperparameter_settings(sampler_file):
    hyp_list = []
    
    with open(sampler_file, "rb") as obj:
        for i in range(10):
            hyp_list.append(pickle.load(obj))
            
    return hyp_list

#returns VGG16 base_model
def load_base_model_vgg16(img_width,img_height,num_channels,weight_init='imagenet',include_fc_layers=False):
    base_model = applications.VGG16(weights=weight_init, input_shape=(img_width, img_height, num_channels)
                                    ,include_top=include_fc_layers)
    return base_model

#hyperparam_setting - dictionary of hyperparameter setting
#base_model - the base model to fine tune
#returns the full model after adding the FC layers to the base_model

def add_fc_layers(hyperparam_setting, base_model):
    num_layers = hyperparam_setting['layers']
    nodes_fc_layer_1 = int(hyperparam_setting['nodes_1'])
    
    print('Dense('+str(nodes_fc_layer_1)+'), use_bias = False)(X)')
    print('BatchNormalization()(X)')
    print('Activation(relu)(X)')
    print('Dropout(0.3)(X)')
    
    X = base_model.output
    X = Flatten(input_shape=base_model.output_shape[1:])(X)
    
    X = Dense(nodes_fc_layer_1, use_bias = False)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = Dropout(0.3)(X)  
    
    if num_layers > 1:
        print('more than one layer')
        
        for n_layer in range(2,num_layers+1):
            #str(setting['nodes_'+str(n_layer)
            
            num_nodes = int(hyperparam_setting['nodes_'+str(n_layer)])
            X = Dense(num_nodes, use_bias = False)(X)
            X = BatchNormalization()(X)
            X = Activation('relu')(X)
            X = Dropout(0.3)(X)  
            
            print('Dense('+str(num_nodes)+'), use_bias = False)(X)')
            print('BatchNormalization()(X)')
            print('Activation(relu)(X)')
            print('Dropout(0.3)(X)')
    else:
        print('only one layer')
        
    Output = Dense(1, use_bias = False)(X) #number of dense nodes is 1 because it is a binary classification problem
    Output = BatchNormalization()(Output)
    Output = Activation('sigmoid')(Output)
    
    model = Model(inputs=base_model.input, outputs=Output)
    return model

#set the first num_layers to nontrainable
#model - an instance of Keras Model
# => model is the final model (base_model added with fully connected layers)

def set_nontrainable_layers(num_layers, model):
    for layer in model.layers[:num_layers]:
        layer.trainable = False
        
    return model

#model - an instance of Keras Model
# => model is the final model (base_model added with fully connected layers)

def add_loss_function(hyperparam_setting, model, metric = 'accuracy'):
    learning_rate = hyperparam_setting['learning_rate']
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(lr = learning_rate),
                  metrics=[metric])
    
    return model

#returns the dict of cross validation settings

def load_cross_validation_settings(cv_file):
    cv_setting = None
    with open(cv_file, "rb") as obj:
        cv_setting = pickle.load(obj)
        
    return cv_setting
    
    
    
def fit_generator(hyperparameter_setting, training_generator, 
                  model, setting_count, cv_fold_idx,
                  save_best_model_only=False):
    #print("==========")

    num_epochs = hyperparameter_setting['epochs']
    history = model.fit(
                training_generator,
                epochs = num_epochs,
                verbose=1
                )
    
    #print(datetime.datetime.now())
    
   # del checkpointer
    return history


def predict_data(model_list, cv_file, 
                         generator_params_dict):
        
        cv_setting = load_cross_validation_settings(cv_file)
        final_test = cv_setting['Test']
        subject_dict_test = {}
        subject_list_test = []
        
        
        for x,y in zip(final_test[0], final_test[1]):
            subject_list_test.append(x)
            subject_dict_test[x] = y
        
        testing_generator = DataGenerator(subject_list_test, subject_dict_test, **generator_params_dict)
        
        prediction_array = []
        accuracy_list = []
        confusion_matrix_list = []
        sensitivity_list = []
        specificity_list = []
        f1_score_list = []
        
        for i in model_list:
            print(i)
            model = load_model(i)
            cur_pred = model.predict_generator(testing_generator, verbose = 1)
            cur_pred_thres = []
            pred_list = []
            
            for pred in cur_pred:
                v = pred[0]
                pred_list.append(v)
                
                if v >= 0.5:
                    cur_pred_thres.append(1)
                else:
                    cur_pred_thres.append(0)
            prediction_array.append(pred_list)
            
            
            y_pred = np.array(cur_pred_thres)
            y_true = np.array(list(subject_dict_test.values()))
            acc = accuracy_score(y_pred, y_true)
            conf = confusion_matrix(y_pred, y_true)
            sen = conf[0,0]/(conf[0,0] + conf[1,0])
            spec = conf[1,1]/(conf[1,1] + conf[0,1])
            f1 = f1_score(y_pred, y_true)
            
            print('sen',sen)
            print('spec', spec)
            print('f1',f1)
            
            accuracy_list.append(acc)
            confusion_matrix_list.append(conf)
            sensitivity_list.append(sen)
            specificity_list.append(spec)
            f1_score_list.append(f1)
            
           
            
        return [prediction_array, accuracy_list, confusion_matrix_list, sensitivity_list, specificity_list, f1_score_list]
img_width = 256
img_height = 256
num_channels = 3

history_path = r'/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/CN_vs_AD/history'
cv_file = '/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_CN_vs_AD_CV_File.pickle'
model_list_path = r'/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/CN_vs_AD/final_models'
model_list = ['top_model_1.hdf5','top_model_2.hdf5','top_model_3.hdf5','top_model_4.hdf5','top_model_5.hdf5','top_model_6.hdf5'
             ,'top_model_7.hdf5','top_model_8.hdf5','top_model_9.hdf5']
full_model_list = []

for i in model_list:
    full_model_list.append(model_list_path + '/' + i)
    
params_dict = {'dim':(img_width,img_height),
                   'batch_size': 1,
                   'n_classes': 2,
                   'n_channels': num_channels,
                   'shuffle':False
                  }
result_list = predict_data(full_model_list, cv_file, params_dict)


/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/CN_vs_AD/final_models/top_model_1.hdf5
Instructions for updating:
Please use Model.predict, which supports generators.
2656/2656 [==============================] - 13s 5ms/step
sen 0.8505825908156271
spec 0.8304093567251462
f1 0.8252386882523868
/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/CN_vs_AD/final_models/top_model_2.hdf5
2656/2656 [==============================] - 14s 5ms/step
sen 0.9026730637422893
spec 0.7844611528822055
f1 0.8244073748902546
/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/CN_vs_AD/final_models/top_model_3.hdf5
2656/2656 [==============================] - 14s 5ms/step
sen 0.7957505140507197
spec 0.8563074352548037
f1 0.8134920634920635
/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/CN_vs_AD/final_models/top_model_4.hdf5
2656/2656 [==============================] - 14s 5ms/step
sen 0.8368745716244003
spec 0.741854636591478

In [29]:
result_list[1]

[0.8414909638554217,
 0.8493975903614458,
 0.8230421686746988,
 0.7940512048192772,
 0.8049698795180723,
 0.8140060240963856,
 0.8456325301204819,
 0.7797439759036144,
 0.8049698795180723]

model averaging

In [3]:
p = result_list[0]

In [7]:
a = np.array(p)
a.shape

(9, 2656)

In [11]:
aT = a.T

In [12]:
aT.shape

(2656, 9)

top-3

In [16]:
aT3 = aT[:, 0:3]

In [17]:
aT3.shape

(2656, 3)

In [23]:
p_avg = np.average(aT3, axis = 1)

In [24]:
p_avg.shape

(2656,)

In [25]:
super_threshold_indices = p_avg  > 0.5
p_avg[super_threshold_indices] = 1
super_threshold_indices = p_avg  <= 0.5
p_avg[super_threshold_indices] = 0

In [26]:
cv_setting = load_cross_validation_settings(cv_file)
final_test = cv_setting['Test']
subject_dict_test = {}
subject_list_test = []
        
        
for x,y in zip(final_test[0], final_test[1]):
    subject_list_test.append(x)
    subject_dict_test[x] = y




In [27]:
accuracy_score(p_avg, np.array(list(subject_dict_test.values())))

0.8505271084337349

In [30]:
conf  = confusion_matrix(p_avg, np.array(list(subject_dict_test.values())))

In [31]:
print('sen',(conf[0,0]/(conf[0,0] + conf[1,0])))
print('spec',(conf[1,1]/(conf[1,1] + conf[0,1])))

sen 0.8732008224811515
spec 0.822890559732665


In [33]:
f1_score(p_avg, np.array(list(subject_dict_test.values())))

0.8322771440642164

top-5


In [34]:
aT5 = aT[:, 0:5]
print(aT5.shape)
p_avg = np.average(aT5, axis = 1)
print(p_avg.shape)

super_threshold_indices = p_avg  > 0.5
p_avg[super_threshold_indices] = 1
super_threshold_indices = p_avg  <= 0.5
p_avg[super_threshold_indices] = 0

cv_setting = load_cross_validation_settings(cv_file)
final_test = cv_setting['Test']
subject_dict_test = {}
subject_list_test = []
for x,y in zip(final_test[0], final_test[1]):
    subject_list_test.append(x)
    subject_dict_test[x] = y




print('acc',accuracy_score(p_avg, np.array(list(subject_dict_test.values()))))
conf  = confusion_matrix(p_avg, np.array(list(subject_dict_test.values())))
print('sen',(conf[0,0]/(conf[0,0] + conf[1,0])))
print('spec',(conf[1,1]/(conf[1,1] + conf[0,1])))

f1_score(p_avg, np.array(list(subject_dict_test.values())))

(2656, 5)
(2656,)
acc 0.8527861445783133
sen 0.8732008224811515
spec 0.8279030910609858


0.8352296670880742

top-7

In [35]:
aT7 = aT[:, 0:7]
print(aT7.shape)
p_avg = np.average(aT7, axis = 1)
print(p_avg.shape)

super_threshold_indices = p_avg  > 0.5
p_avg[super_threshold_indices] = 1
super_threshold_indices = p_avg  <= 0.5
p_avg[super_threshold_indices] = 0

cv_setting = load_cross_validation_settings(cv_file)
final_test = cv_setting['Test']
subject_dict_test = {}
subject_list_test = []
for x,y in zip(final_test[0], final_test[1]):
    subject_list_test.append(x)
    subject_dict_test[x] = y




print('acc',accuracy_score(p_avg, np.array(list(subject_dict_test.values()))))
conf  = confusion_matrix(p_avg, np.array(list(subject_dict_test.values())))
print('sen',(conf[0,0]/(conf[0,0] + conf[1,0])))
print('spec',(conf[1,1]/(conf[1,1] + conf[0,1])))

f1_score(p_avg, np.array(list(subject_dict_test.values())))

(2656, 7)
(2656,)
acc 0.8471385542168675
sen 0.8704592186429061
spec 0.8187134502923976


0.8284023668639053

top-9

In [37]:
aT9 = aT[:, 0:9]
print(aT9.shape)
p_avg = np.average(aT9, axis = 1)
print(p_avg.shape)

super_threshold_indices = p_avg  > 0.5
p_avg[super_threshold_indices] = 1
super_threshold_indices = p_avg  <= 0.5
p_avg[super_threshold_indices] = 0

cv_setting = load_cross_validation_settings(cv_file)
final_test = cv_setting['Test']
subject_dict_test = {}
subject_list_test = []
for x,y in zip(final_test[0], final_test[1]):
    subject_list_test.append(x)
    subject_dict_test[x] = y




print('acc',accuracy_score(p_avg, np.array(list(subject_dict_test.values()))))
conf  = confusion_matrix(p_avg, np.array(list(subject_dict_test.values())))
print('sen',(conf[0,0]/(conf[0,0] + conf[1,0])))
print('spec',(conf[1,1]/(conf[1,1] + conf[0,1])))

f1_score(p_avg, np.array(list(subject_dict_test.values())))

(2656, 9)
(2656,)
acc 0.8396084337349398
sen 0.9013022618231665
spec 0.7644110275689223


0.8111702127659574

top-4

In [38]:
aT4 = aT[:, 0:4]
print(aT4.shape)
p_avg = np.average(aT4, axis = 1)
print(p_avg.shape)

super_threshold_indices = p_avg  > 0.5
p_avg[super_threshold_indices] = 1
super_threshold_indices = p_avg  <= 0.5
p_avg[super_threshold_indices] = 0

cv_setting = load_cross_validation_settings(cv_file)
final_test = cv_setting['Test']
subject_dict_test = {}
subject_list_test = []
for x,y in zip(final_test[0], final_test[1]):
    subject_list_test.append(x)
    subject_dict_test[x] = y




print('acc',accuracy_score(p_avg, np.array(list(subject_dict_test.values()))))
conf  = confusion_matrix(p_avg, np.array(list(subject_dict_test.values())))
print('sen',(conf[0,0]/(conf[0,0] + conf[1,0])))
print('spec',(conf[1,1]/(conf[1,1] + conf[0,1])))

f1_score(p_avg, np.array(list(subject_dict_test.values())))

(2656, 4)
(2656,)
acc 0.8505271084337349
sen 0.8779986291980809
spec 0.8170426065162907


0.8312792180195494

top-6

In [39]:
aT6 = aT[:, 0:6]
print(aT6.shape)
p_avg = np.average(aT6, axis = 1)
print(p_avg.shape)

super_threshold_indices = p_avg  > 0.5
p_avg[super_threshold_indices] = 1
super_threshold_indices = p_avg  <= 0.5
p_avg[super_threshold_indices] = 0

cv_setting = load_cross_validation_settings(cv_file)
final_test = cv_setting['Test']
subject_dict_test = {}
subject_list_test = []
for x,y in zip(final_test[0], final_test[1]):
    subject_list_test.append(x)
    subject_dict_test[x] = y




print('acc',accuracy_score(p_avg, np.array(list(subject_dict_test.values()))))
conf  = confusion_matrix(p_avg, np.array(list(subject_dict_test.values())))
print('sen',(conf[0,0]/(conf[0,0] + conf[1,0])))
print('spec',(conf[1,1]/(conf[1,1] + conf[0,1])))

f1_score(p_avg, np.array(list(subject_dict_test.values())))

(2656, 6)
(2656,)
acc 0.848644578313253
sen 0.8690884167237835
spec 0.8237259816207184


0.8306655433866892

top-8

In [40]:
aT8 = aT[:, 0:8]
print(aT8.shape)
p_avg = np.average(aT8, axis = 1)
print(p_avg.shape)

super_threshold_indices = p_avg  > 0.5
p_avg[super_threshold_indices] = 1
super_threshold_indices = p_avg  <= 0.5
p_avg[super_threshold_indices] = 0

cv_setting = load_cross_validation_settings(cv_file)
final_test = cv_setting['Test']
subject_dict_test = {}
subject_list_test = []
for x,y in zip(final_test[0], final_test[1]):
    subject_list_test.append(x)
    subject_dict_test[x] = y




print('acc',accuracy_score(p_avg, np.array(list(subject_dict_test.values()))))
conf  = confusion_matrix(p_avg, np.array(list(subject_dict_test.values())))
print('sen',(conf[0,0]/(conf[0,0] + conf[1,0])))
print('spec',(conf[1,1]/(conf[1,1] + conf[0,1])))

f1_score(p_avg, np.array(list(subject_dict_test.values())))

(2656, 8)
(2656,)
acc 0.848644578313253
sen 0.8697738176833447
spec 0.822890559732665


0.8305227655986509

max voting

In [1]:
import pickle
import pandas as pd
import datetime
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import applications
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from DataGen_BinaryLabels import DataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import backend
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, f1_score

#returns a list  of  hyperparamter settings
def load_hyperparameter_settings(sampler_file):
    hyp_list = []
    
    with open(sampler_file, "rb") as obj:
        for i in range(10):
            hyp_list.append(pickle.load(obj))
            
    return hyp_list

#returns VGG16 base_model
def load_base_model_vgg16(img_width,img_height,num_channels,weight_init='imagenet',include_fc_layers=False):
    base_model = applications.VGG16(weights=weight_init, input_shape=(img_width, img_height, num_channels)
                                    ,include_top=include_fc_layers)
    return base_model

#hyperparam_setting - dictionary of hyperparameter setting
#base_model - the base model to fine tune
#returns the full model after adding the FC layers to the base_model

def add_fc_layers(hyperparam_setting, base_model):
    num_layers = hyperparam_setting['layers']
    nodes_fc_layer_1 = int(hyperparam_setting['nodes_1'])
    
    print('Dense('+str(nodes_fc_layer_1)+'), use_bias = False)(X)')
    print('BatchNormalization()(X)')
    print('Activation(relu)(X)')
    print('Dropout(0.3)(X)')
    
    X = base_model.output
    X = Flatten(input_shape=base_model.output_shape[1:])(X)
    
    X = Dense(nodes_fc_layer_1, use_bias = False)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = Dropout(0.3)(X)  
    
    if num_layers > 1:
        print('more than one layer')
        
        for n_layer in range(2,num_layers+1):
            #str(setting['nodes_'+str(n_layer)
            
            num_nodes = int(hyperparam_setting['nodes_'+str(n_layer)])
            X = Dense(num_nodes, use_bias = False)(X)
            X = BatchNormalization()(X)
            X = Activation('relu')(X)
            X = Dropout(0.3)(X)  
            
            print('Dense('+str(num_nodes)+'), use_bias = False)(X)')
            print('BatchNormalization()(X)')
            print('Activation(relu)(X)')
            print('Dropout(0.3)(X)')
    else:
        print('only one layer')
        
    Output = Dense(1, use_bias = False)(X) #number of dense nodes is 1 because it is a binary classification problem
    Output = BatchNormalization()(Output)
    Output = Activation('sigmoid')(Output)
    
    model = Model(inputs=base_model.input, outputs=Output)
    return model

#set the first num_layers to nontrainable
#model - an instance of Keras Model
# => model is the final model (base_model added with fully connected layers)

def set_nontrainable_layers(num_layers, model):
    for layer in model.layers[:num_layers]:
        layer.trainable = False
        
    return model

#model - an instance of Keras Model
# => model is the final model (base_model added with fully connected layers)

def add_loss_function(hyperparam_setting, model, metric = 'accuracy'):
    learning_rate = hyperparam_setting['learning_rate']
    
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(lr = learning_rate),
                  metrics=[metric])
    
    return model

#returns the dict of cross validation settings

def load_cross_validation_settings(cv_file):
    cv_setting = None
    with open(cv_file, "rb") as obj:
        cv_setting = pickle.load(obj)
        
    return cv_setting
    
    
    
def fit_generator(hyperparameter_setting, training_generator, 
                  model, setting_count, cv_fold_idx,
                  save_best_model_only=False):
    #print("==========")

    num_epochs = hyperparameter_setting['epochs']
    history = model.fit(
                training_generator,
                epochs = num_epochs,
                verbose=1
                )
    
    #print(datetime.datetime.now())
    
   # del checkpointer
    return history


def predict_data(model_list, cv_file, 
                         generator_params_dict):
        
        cv_setting = load_cross_validation_settings(cv_file)
        final_test = cv_setting['Test']
        subject_dict_test = {}
        subject_list_test = []
        
        
        for x,y in zip(final_test[0], final_test[1]):
            subject_list_test.append(x)
            subject_dict_test[x] = y
        
        testing_generator = DataGenerator(subject_list_test, subject_dict_test, **generator_params_dict)
        
        prediction_array = []
        accuracy_list = []
        confusion_matrix_list = []
        sensitivity_list = []
        specificity_list = []
        f1_score_list = []
        
        for i in model_list:
            print(i)
            model = load_model(i)
            cur_pred = model.predict_generator(testing_generator, verbose = 1)
            cur_pred_thres = []
            pred_list = []
            
            for pred in cur_pred:
                v = pred[0]
                pred_list.append(v)
                
                if v >= 0.5:
                    cur_pred_thres.append(1)
                else:
                    cur_pred_thres.append(0)
            prediction_array.append(pred_list)
            
            
            y_pred = np.array(cur_pred_thres)
            y_true = np.array(list(subject_dict_test.values()))
            acc = accuracy_score(y_pred, y_true)
            conf = confusion_matrix(y_pred, y_true)
            sen = conf[0,0]/(conf[0,0] + conf[1,0])
            spec = conf[1,1]/(conf[1,1] + conf[0,1])
            f1 = f1_score(y_pred, y_true)
            
            print('sen',sen)
            print('spec', spec)
            print('f1',f1)
            
            accuracy_list.append(acc)
            confusion_matrix_list.append(conf)
            sensitivity_list.append(sen)
            specificity_list.append(spec)
            f1_score_list.append(f1)
            
           
            
        return [prediction_array, accuracy_list, confusion_matrix_list, sensitivity_list, specificity_list, f1_score_list]
img_width = 256
img_height = 256
num_channels = 3

history_path = r'/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/CN_vs_AD/history'
cv_file = '/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_CN_vs_AD_CV_File.pickle'
model_list_path = r'/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/CN_vs_AD/final_models'
model_list = ['top_model_1.hdf5','top_model_2.hdf5','top_model_3.hdf5','top_model_4.hdf5','top_model_5.hdf5','top_model_6.hdf5'
             ,'top_model_7.hdf5','top_model_8.hdf5','top_model_9.hdf5']
full_model_list = []

for i in model_list:
    full_model_list.append(model_list_path + '/' + i)
    
params_dict = {'dim':(img_width,img_height),
                   'batch_size': 1,
                   'n_classes': 2,
                   'n_channels': num_channels,
                   'shuffle':False
                  }
result_list = predict_data(full_model_list, cv_file, params_dict)


/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/CN_vs_AD/final_models/top_model_1.hdf5
Instructions for updating:
Please use Model.predict, which supports generators.
2656/2656 [==============================] - 55s 21ms/step
sen 0.8505825908156271
spec 0.8304093567251462
f1 0.8252386882523868
/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/CN_vs_AD/final_models/top_model_2.hdf5
2656/2656 [==============================] - 14s 5ms/step
sen 0.9026730637422893
spec 0.7844611528822055
f1 0.8244073748902546
/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/CN_vs_AD/final_models/top_model_3.hdf5
2656/2656 [==============================] - 14s 5ms/step
sen 0.7957505140507197
spec 0.8563074352548037
f1 0.8134920634920635
/media/iitindmaths/Seagate_Expansion_Drive/DTE_ADNI_Baseline_Revision/CN_vs_AD/final_models/top_model_4.hdf5
2656/2656 [==============================] - 15s 5ms/step
sen 0.8368745716244003
spec 0.74185463659147

top-3

In [2]:
p = result_list[0]
a = np.array(p)
a.shape

(9, 2656)

In [3]:
aT = a.T
aT.shape

(2656, 9)

In [4]:
aT3 = aT[:, 0:3]
print(aT3.shape)

super_threshold_indices = aT3  > 0.5
aT3[super_threshold_indices] = 1
super_threshold_indices = aT3  <= 0.5
aT3[super_threshold_indices] = 0



(2656, 3)


In [5]:
nz_count = np.count_nonzero(aT3, axis = 1)

In [6]:
y_pred = []
for nzc in nz_count:
    if nzc >= 2:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [7]:
y_pred_arr = np.array(y_pred)

In [8]:
cv_setting = load_cross_validation_settings(cv_file)
final_test = cv_setting['Test']
subject_dict_test = {}
subject_list_test = []
for x,y in zip(final_test[0], final_test[1]):
    subject_list_test.append(x)
    subject_dict_test[x] = y


In [9]:

print('acc',accuracy_score(y_pred_arr, np.array(list(subject_dict_test.values()))))
conf  = confusion_matrix(y_pred_arr, np.array(list(subject_dict_test.values())))
print('sen',(conf[0,0]/(conf[0,0] + conf[1,0])))
print('spec',(conf[1,1]/(conf[1,1] + conf[0,1])))

f1_score(y_pred_arr, np.array(list(subject_dict_test.values())))

acc 0.8475150602409639
sen 0.8636052090472927
spec 0.8279030910609858


0.8303309593632174

top-5

In [10]:
aT5 = aT[:, 0:5]
print(aT5.shape)

super_threshold_indices = aT5  > 0.5
aT5[super_threshold_indices] = 1
super_threshold_indices = aT5  <= 0.5
aT5[super_threshold_indices] = 0

nz_count = np.count_nonzero(aT5, axis = 1)
y_pred = []
for nzc in nz_count:
    if nzc >= 3:
        y_pred.append(1)
    else:
        y_pred.append(0)

y_pred_arr = np.array(y_pred)
cv_setting = load_cross_validation_settings(cv_file)
final_test = cv_setting['Test']
subject_dict_test = {}
subject_list_test = []
for x,y in zip(final_test[0], final_test[1]):
    subject_list_test.append(x)
    subject_dict_test[x] = y

print('acc',accuracy_score(y_pred_arr, np.array(list(subject_dict_test.values()))))
conf  = confusion_matrix(y_pred_arr, np.array(list(subject_dict_test.values())))
print('sen',(conf[0,0]/(conf[0,0] + conf[1,0])))
print('spec',(conf[1,1]/(conf[1,1] + conf[0,1])))
f1_score(y_pred_arr, np.array(list(subject_dict_test.values())))


(2656, 5)
acc 0.8414909638554217
sen 0.8560657984921178
spec 0.8237259816207184


0.8240702047638946

top-7

In [11]:
aT7 = aT[:, 0:7]
print(aT5.shape)

super_threshold_indices = aT7  > 0.5
aT7[super_threshold_indices] = 1
super_threshold_indices = aT7  <= 0.5
aT7[super_threshold_indices] = 0

nz_count = np.count_nonzero(aT7, axis = 1)
y_pred = []
for nzc in nz_count:
    if nzc >= 4:
        y_pred.append(1)
    else:
        y_pred.append(0)

y_pred_arr = np.array(y_pred)
cv_setting = load_cross_validation_settings(cv_file)
final_test = cv_setting['Test']
subject_dict_test = {}
subject_list_test = []
for x,y in zip(final_test[0], final_test[1]):
    subject_list_test.append(x)
    subject_dict_test[x] = y

print('acc',accuracy_score(y_pred_arr, np.array(list(subject_dict_test.values()))))
conf  = confusion_matrix(y_pred_arr, np.array(list(subject_dict_test.values())))
print('sen',(conf[0,0]/(conf[0,0] + conf[1,0])))
print('spec',(conf[1,1]/(conf[1,1] + conf[0,1])))
f1_score(y_pred_arr, np.array(list(subject_dict_test.values())))


(2656, 5)
acc 0.8501506024096386
sen 0.8677176148046607
spec 0.8287385129490392


0.832913518052057

top-9

In [12]:
aT9 = aT[:, 0:9]
print(aT9.shape)

super_threshold_indices = aT9  > 0.5
aT9[super_threshold_indices] = 1
super_threshold_indices = aT9  <= 0.5
aT9[super_threshold_indices] = 0

nz_count = np.count_nonzero(aT9, axis = 1)
y_pred = []
for nzc in nz_count:
    if nzc >= 5:
        y_pred.append(1)
    else:
        y_pred.append(0)

y_pred_arr = np.array(y_pred)
cv_setting = load_cross_validation_settings(cv_file)
final_test = cv_setting['Test']
subject_dict_test = {}
subject_list_test = []
for x,y in zip(final_test[0], final_test[1]):
    subject_list_test.append(x)
    subject_dict_test[x] = y

print('acc',accuracy_score(y_pred_arr, np.array(list(subject_dict_test.values()))))
conf  = confusion_matrix(y_pred_arr, np.array(list(subject_dict_test.values())))
print('sen',(conf[0,0]/(conf[0,0] + conf[1,0])))
print('spec',(conf[1,1]/(conf[1,1] + conf[0,1])))
f1_score(y_pred_arr, np.array(list(subject_dict_test.values())))


(2656, 9)
acc 0.8475150602409639
sen 0.8725154215215901
spec 0.8170426065162907


0.8284625158831004